#### Import packages

In [ ]:
# libraries
import pandas as pd
import pydbtools as pydb
import boto3
from datetime import datetime
#import re
import os  # for file paths
import awswrangler as wr

In [ ]:
# Structured  dataframes
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

#### Assigning key variables

In [ ]:
# This is the database where the familyman data is stored on the platform
db1 = "familyman_dev_v3"
# This database is used for FCCSQ processing
fcsq_db = "fcsq"
# create path for within athena FCSQ database in the S3 folder, alongside other S3 items
fcsq_db_path = f"s3://alpha-family-data/fcsq_processing/CA_disps/"
s3 = boto3.resource("s3")
bucket = s3.Bucket("alpha-family-data")
#Select the snapshot date you require. Each record in Familyman has a snapshot date (essentially just a column in the tables) that relates to the database at that point in time
snapshot_date = "2022-08-04"

### Private Law Timeliness

In [ ]:
#Date of issue for initial application in case - as the DOI isn't linked to the U22 event the U22 really just helps to get to one record per case, with the date of issue being the earliest
   #one recorded on the case
pydb.create_temp_table(
f"""
SELECT
  case_number,
  MIN(Case_DOI) AS issue_date
FROM
   {fcsq_db}.ca_apps_issue_date
WHERE
  field_model = 'U22_AT'
  AND order_case_type = 'P'
GROUP BY
  case_number  

""",

"ca_apps_doi")

In [ ]:
#get the last disposal date in the case, filtering by closed cases only
pydb.create_temp_table(
f"""
SELECT
  c.case_number,
  d.first_close_date
FROM
  {fcsq_db}.ca_closed_cases c
  LEFT JOIN {fcsq_db}.ca_closed_date_events d
    ON c.case_number = d.case_number
WHERE 
  c.case_type = 'P'
""",

"ca_first_close")

In [ ]:
#get the last disposal date in the case, filtering by closed cases only
pydb.create_temp_table(
f"""
SELECT
  f.case_number,
  f.first_close_date,
  MAX(c.disp_date) AS last_disposal_date
FROM
  __temp__.ca_first_close f
  LEFT JOIN {fcsq_db}.ca_disps_all_children c
    ON f.case_number = c.case_number
WHERE 
  c.disp_date <= first_close_date
GROUP BY
  f.case_number,
  f.first_close_date
""",

"ca_last_disp_date")

In [ ]:
#get the last disposal date in the case, filtering by closed cases only
pydb.create_temp_table(
f"""
SELECT
  s.case_number,
  s.issue_date,
  c.last_disposal_date,
  date_diff('day',s.issue_date, c.last_disposal_date) AS issue_to_last_disp_days,
  EXTRACT(YEAR FROM c.last_disposal_date) AS disp_year,
  EXTRACT(QUARTER FROM c.last_disposal_date) AS disp_quarter
FROM
  __temp__.ca_apps_doi s
  INNER JOIN __temp__.ca_last_disp_date c
    ON s.case_number = c.case_number
WHERE
  date_diff('day',s.issue_date, c.last_disposal_date) >= 0
    
""",

"ca_issue_to_last_disp")

In [ ]:
pydb.read_sql_query ("select disp_year, disp_quarter, count(*) as count, AVG(issue_to_last_disp_days)/7 AS av_days FROM __temp__.ca_issue_to_last_disp where disp_year = 2011 and disp_quarter = 4 group by disp_year, disp_quarter")

In [ ]:
data = pydb.read_sql_query ("select * FROM __temp__.ca_issue_to_last_disp where disp_year = 2011 and disp_quarter = 4")

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.to_csv ('Exports/t9_case_check.csv', header = True)

In [ ]:
pydb.read_sql_query(f"""select * from {fcsq_db}.ca_apps_child where case_number = 'AF11P50004'""")